In [1]:
! pip install nltk textblob deep_translator pandas scrapy

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from deep_translator import GoogleTranslator
from textblob import TextBlob

# Descargar los recursos necesarios de NLTK
nltk.download('punkt')
nltk.download('stopwords')

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to /home/md/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/md/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import threading

count = 0

stop_words = set(stopwords.words('english'))

def filtrar_texto(texto):
    palabras = word_tokenize(texto)
    palabras_filtradas = [palabra for palabra in palabras if palabra not in stop_words and TextBlob(palabra).sentiment.polarity != 0]
    return ' '.join(palabras_filtradas)

def dividir_texto(texto, limite_caracteres=4000):
    palabras = texto.split()
    secciones = []
    current_section = palabras[0]

    for palabra in palabras[1:]:
        if len(current_section) + len(palabra) + 1 <= limite_caracteres:
            current_section += ' ' + palabra
        else:
            secciones.append(current_section)
            current_section = palabra

    secciones.append(current_section)
    return secciones

def translate(seccion, translated_text_list):
    translated_text_list.append(GoogleTranslator(source='auto', target='en').translate(seccion))

def preprocesar_texto(discurso):
    global count
    print(f"i = {count}")
    count += 1
    secciones = dividir_texto(discurso)
    threads = []
    lista_traducida = []
    for seccion in secciones:
        thread = threading.Thread(target=translate, args=(seccion, lista_traducida))
        threads.append(thread)
        thread.start()

    for thread in threads:
        thread.join()

    texto_traducido = ' '.join(lista_traducida)
    return filtrar_texto(texto_traducido)

In [4]:
import pandas as pd
df = pd.read_csv("discursos.csv")

h = df.head(5).copy()

In [5]:
h["palabras_claves"] = h["discurso"].apply(preprocesar_texto)

i = 0
i = 1
i = 2
i = 3
i = 4


In [6]:
'''
Archivo obtenido de:
https://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm

The NRC Emotion Lexicon is a list of English words and their associations with eight basic emotions 
(anger, fear, anticipation, trust, surprise, sadness, joy, and disgust) and two sentiments (negative and positive). 
The annotations were manually done by crowdsourcing.
'''

NRC_LEXICON = 'NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'

emolex = {}
with open(NRC_LEXICON, 'r') as f:
    for line in f:
        word, emotion, flag = line.strip().split()
        if word not in emolex:
            emolex[word] = {}
        emolex[word][emotion] = int(flag)

def calcular_emociones(texto_preprocesado):
    emotions = {emotion: 0 for emotion in emolex[list(emolex.keys())[0]].keys()}
    for word in texto_preprocesado.split():
        if word in emolex:
            for emotion in emolex[word]:
                if emolex[word][emotion] == 1:
                    emotions[emotion] += emolex[word][emotion]
    return emotions

In [7]:
def emotions(discurso):
    emociones = calcular_emociones(discurso)
    return emociones

In [8]:
for index, row in h.iterrows():
    emociones = emotions(row["palabras_claves"])
    for emocion, valor in emociones.items():
        h[emocion] = valor
    
    print("-" * 30)

Resultado para fila 1:
------------------------------
Resultado para fila 2:
------------------------------
Resultado para fila 3:
------------------------------
Resultado para fila 4:
------------------------------
Resultado para fila 5:
------------------------------


In [9]:
    h

,orador,fecha,discurso,palabras_claves,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
0,Alberto Fernández,05/10/2023,de hablar. Yo les propongo un poquito reflexi...,insulting live drunk famous Honestly full bett...,0,0,0,0,0,0,2,0,0,2
1,Alberto Fernández,30/08/2023,Buen día a todos y todas. ¿Cómo les va? ¿Cómo ...,Good happy love kind cruel beautiful old kind ...,0,0,0,0,0,0,2,0,0,2
2,Alberto Fernández,06/10/2023,"Palabras del presidente de la Nación, Alberto ...",new first good welcome joy particular characte...,0,0,0,0,0,0,2,0,0,2
3,Alberto Fernández,05/10/2023,Buenos a todos y a todas. Gracias a quienes no...,experienced ethical good bad mental young whol...,0,0,0,0,0,0,2,0,0,2
4,Alberto Fernández,30/08/2023,Hay dos cosas que quería contarte que pasaron ...,important first closed hard important economic...,0,0,0,0,0,0,2,0,0,2
